In [71]:
%store -r full_df
full_df.sort_values(by = "player_name_x")

,player_name_x,player_id,receptions,targets,receiving_yards,pass_ep,receiving_touchdown,td_prob_x,receiver_air_yards,yards_after_catch,...,lag_xpass_3,lag_pass_oe_3,lag_Position_3,lag_Team_3,lag_fumbles_3,lag_season_3,lag_standard_ffp_3,lag_half_ppr_ffp_3,lag_full_ppr_ffp_3,total_air_yards
0,A.Abdullah,00-0032104,15.0,21.0,88.0,1.940526,1.0,0.309296,0.428571,5.266667,...,0.0,0.0,RB,DET,0.0,2016,21.8,24.3,26.8,9.0
0,A.Abdullah,00-0032104,8.0,9.0,58.0,2.555794,2.0,0.354787,-0.555556,7.875000,...,0.0,0.0,RB,DET,1.0,2017,99.4,111.9,124.4,-5.0
0,A.Abdullah,00-0032104,25.0,35.0,162.0,2.649383,1.0,0.411380,1.514286,5.840000,...,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1.0,-1.0,53.0
0,A.Abdullah,00-0032104,3.0,4.0,28.0,3.209497,0.0,0.408779,4.250000,9.333333,...,0.0,0.0,RB,DET,2.0,2015,92.0,104.5,117.0,17.0
0,A.Abdullah,00-0032104,5.0,5.0,57.0,2.415451,1.0,0.391956,0.600000,10.800000,...,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1.0,-1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,Z.Zenner,00-0031665,2.0,3.0,11.0,1.203276,0.0,0.306370,-2.000000,6.000000,...,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1.0,-1.0,-6.0
521,Z.Zenner,00-0031665,7.0,10.0,56.0,1.906124,0.0,0.328774,0.000000,9.285714,...,0.0,0.0,FB,DET,0.0,2015,7.1,8.1,9.1,0.0
503,Z.Zenner,00-0031665,18.0,23.0,196.0,1.855442,0.0,0.340058,-0.782609,11.388889,...,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1.0,-1.0,-18.0
526,Z.Zenner,00-0031665,2.0,3.0,19.0,1.495626,0.0,0.359905,1.666667,9.500000,...,0.0,0.0,FB,DET,1.0,2016,75.0,84.0,93.0,5.0


In [72]:
full_df = full_df.drop(280)
full_df = full_df.drop(283)



In [73]:
import pandas as pd
full_df["total_air_yards"] = full_df["receiver_air_yards"] * full_df["targets"]

pd.set_option('chained_assignment', None)

lag_features = ["total_air_yards"]

for lag in range(1, 4):

    shifted = full_df.groupby('player_id').shift(lag)
    
    for column in lag_features:
        
        full_df[f'lag_{column}_{lag}'] = shifted[column]
        
full_df = full_df.fillna(0)

In [74]:
import numpy as np

team_df = full_df.groupby(["Team", "season"]).agg({
    "targets" : np.sum,
    "receiving_yards" : np.sum,
    "total_air_yards" : np.sum
})



In [75]:
team_df.sort_values(by = "receiving_yards", ascending = False)

,,targets,receiving_yards,total_air_yards
Team,season,,,
DEN,2013,671.0,5572.0,5258.852113
NO,2011,659.0,5505.0,5070.273973
KC,2018,627.0,5480.0,6131.166667
TB,2018,629.0,5386.0,6580.200578
NO,2016,674.0,5298.0,4875.991228
...,...,...,...,...
CLE,2008,257.0,2463.0,1532.491525
OAK,2020,306.0,2438.0,2178.779310
DEN,2011,391.0,2425.0,4223.000000


In [88]:
full_df.nunique()
#This shows there are more cases like John Brown (duplicates that will mess with the data a tiny bit).
#I don't feel like changing it right now. I'll fix it later.

player_name_x            2189
player_id                2358
receptions                123
targets                   187
receiving_yards          1126
                         ... 
lag_full_ppr_ffp_3       2066
total_air_yards          2040
lag_total_air_yards_1    1835
lag_total_air_yards_2    1538
lag_total_air_yards_3    1235
Length: 146, dtype: int64

In [83]:
vacated_df = full_df.loc[(full_df["lag_Team_1"] != -1)]
vacated_df = vacated_df.loc[(vacated_df["lag_Team_1"] != vacated_df["Team"])]

In [84]:
vacated_df

,player_name_x,player_id,receptions,targets,receiving_yards,pass_ep,receiving_touchdown,td_prob_x,receiver_air_yards,yards_after_catch,...,lag_Team_3,lag_fumbles_3,lag_season_3,lag_standard_ffp_3,lag_half_ppr_ffp_3,lag_full_ppr_ffp_3,total_air_yards,lag_total_air_yards_1,lag_total_air_yards_2,lag_total_air_yards_3
2,A.Becht,00-0019457,6.0,6.0,39.0,1.196796,0.0,0.307513,3.333333,3.166667,...,-1,-1.0,-1,-1.00,-1.00,-1.00,20.0,33.833333,0.0,0.0
9,A.Crumpler,00-0020285,24.0,28.0,257.0,2.249767,1.0,0.357455,8.000000,4.875000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,224.0,198.000000,0.0,0.0
11,A.Fasano,00-0024268,34.0,34.0,454.0,2.397865,7.0,0.352528,9.294118,4.058824,...,-1,-1.0,-1,-1.00,-1.00,-1.00,316.0,89.000000,0.0,0.0
18,A.Lelie,00-0021152,11.0,16.0,197.0,1.617755,2.0,0.320984,14.000000,2.181818,...,-1,-1.0,-1,-1.00,-1.00,-1.00,224.0,106.000000,0.0,0.0
19,A.Pearman,00-0023561,1.0,1.0,23.0,3.557720,1.0,0.356703,21.000000,2.000000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,21.0,-1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,G.Smith,00-0030565,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,NYJ,0.0,2016,9.18,9.18,9.18,0.0,0.000000,0.0,0.0
593,J.Dobbs,00-0033949,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,0.0,0.000000,0.0,0.0
603,J.Natson,00-0033266,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,0.0,85.000000,0.0,0.0
652,B.Colquitt,00-0026516,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,0.0,0.000000,0.0,0.0


In [85]:
vacated_team_df = vacated_df.groupby(["lag_Team_1", "season"]).agg({
    'lag_receiving_yards_1' : np.sum,
    'lag_targets_1' : np.sum,
    'lag_total_air_yards_1' : np.sum      
})

#wr_team_df = wr_team_df.reset_index()

In [89]:
vacated_team_df.sort_values(by = "lag_targets_1", ascending = False).head(60)

,,lag_receiving_yards_1,lag_targets_1,lag_total_air_yards_1
lag_Team_1,season,,,
SD,2017,4309.0,534.0,4767.985798
STL,2016,2627.0,410.0,3403.480000
SEA,2010,2442.0,373.0,3079.000000
PIT,2019,2375.0,314.0,2232.692308
BAL,2019,1958.0,304.0,2890.691681
DEN,2011,2471.0,303.0,4151.660714
LA,2017,2156.0,296.0,2614.000000
STL,2013,2026.0,288.0,2081.187372
NO,2017,2282.0,282.0,2700.000000


In [37]:
"""Top 2 don't count (They are the seasons in which San Diego and St Louis moved to LA so every target is showing
as vacated)"""

TypeError: 'DataFrame' object is not callable

In [94]:
steelers_df = full_df.loc[(full_df["Team"] == "PIT")]
steelers_2018 = steelers_df.loc[(steelers_df["season"] == 2018)]
steelers_2019 = steelers_df.loc[(steelers_df["season"] == 2019)]

In [96]:
steelers_2018.sort_values(by = "targets", ascending = False).loc[""]

,player_name_x,player_id,receptions,targets,receiving_yards,pass_ep,receiving_touchdown,td_prob_x,receiver_air_yards,yards_after_catch,...,lag_Team_3,lag_fumbles_3,lag_season_3,lag_standard_ffp_3,lag_half_ppr_ffp_3,lag_full_ppr_ffp_3,total_air_yards,lag_total_air_yards_1,lag_total_air_yards_2,lag_total_air_yards_3
4,A.Brown,00-0027793,104.0,168.0,1297.0,1.955030,15.0,0.353809,11.255952,4.663462,...,PIT,1.0,2015,244.58,312.58,380.58,1891.000000,2213.834356,1614.857143,2153.730570
274,J.Smith-Schuster,00-0033857,111.0,167.0,1383.0,2.173260,7.0,0.365371,8.801205,5.450450,...,-1,-1.0,-1,-1.00,-1.00,-1.00,1469.801205,729.000000,0.000000,0.000000
220,J.Conner,00-0033553,55.0,73.0,497.0,1.917925,1.0,0.334069,-0.253521,9.963636,...,-1,-1.0,-1,-1.00,-1.00,-1.00,-18.507042,-4.000000,0.000000,0.000000
508,V.McDonald,00-0030470,50.0,72.0,610.0,1.717141,4.0,0.337815,5.277778,7.740000,...,SF,0.0,2015,50.60,65.60,80.60,380.000000,199.000000,439.000000,385.000000
432,R.Switzer,00-0033562,36.0,44.0,253.0,2.154748,1.0,0.373555,3.363636,3.805556,...,-1,-1.0,-1,-1.00,-1.00,-1.00,148.000000,33.000000,0.000000,0.000000
245,J.James,00-0032135,30.0,40.0,423.0,2.031330,2.0,0.371440,7.692308,6.166667,...,PIT,0.0,2015,11.60,15.60,19.60,307.692308,386.000000,400.566667,75.272727
279,J.Washington,00-0034676,16.0,38.0,217.0,1.910521,1.0,0.337400,16.842105,3.312500,...,-1,-1.0,-1,-1.00,-1.00,-1.00,640.000000,0.000000,0.000000,0.000000
272,J.Samuels,00-0034331,26.0,29.0,199.0,2.355237,3.0,0.362735,0.068966,8.269231,...,-1,-1.0,-1,-1.00,-1.00,-1.00,2.000000,0.000000,0.000000,0.000000
182,E.Rogers,00-0031782,12.0,15.0,79.0,1.850635,0.0,0.356090,4.428571,3.333333,...,-1,-1.0,-1,-1.00,-1.00,-1.00,66.428571,270.000000,623.000000,0.000000
243,J.Hunter,00-0030521,3.0,13.0,21.0,1.911027,0.0,0.319691,20.461538,0.666667,...,TEN,0.0,2015,32.40,43.40,54.40,266.000000,124.000000,341.000000,377.000000


In [97]:
steelers_2019.sort_values(by = "targets", ascending = False)

,player_name_x,player_id,receptions,targets,receiving_yards,pass_ep,receiving_touchdown,td_prob_x,receiver_air_yards,yards_after_catch,...,lag_Team_3,lag_fumbles_3,lag_season_3,lag_standard_ffp_3,lag_half_ppr_ffp_3,lag_full_ppr_ffp_3,total_air_yards,lag_total_air_yards_1,lag_total_air_yards_2,lag_total_air_yards_3
135,D.Johnson,00-0035216,59.0,92.0,680.0,1.583293,5.0,0.321195,9.163043,5.169492,...,-1,-1.0,-1,-1.00,-1.00,-1.00,843.000000,0.000000,0.0,0.0
275,J.Smith-Schuster,00-0033857,42.0,71.0,552.0,1.629773,3.0,0.335111,9.757143,5.285714,...,-1,-1.0,-1,-1.00,-1.00,-1.00,692.757143,1469.801205,729.0,0.0
270,J.Samuels,00-0034331,47.0,57.0,305.0,1.875475,1.0,0.317993,-0.789474,7.787234,...,-1,-1.0,-1,-1.00,-1.00,-1.00,-45.000000,2.000000,0.0,0.0
507,V.McDonald,00-0030470,38.0,55.0,273.0,1.600581,3.0,0.327829,5.054545,4.526316,...,SF,0.0,2016,63.10,75.10,87.10,278.000000,380.000000,199.0,439.0
227,J.Conner,00-0033553,34.0,38.0,251.0,2.044451,3.0,0.369453,-1.421053,8.764706,...,-1,-1.0,-1,-1.00,-1.00,-1.00,-54.000000,-18.507042,-4.0,0.0
384,N.Vannett,00-0032394,17.0,22.0,166.0,1.407090,0.0,0.310103,6.363636,3.882353,...,SEA,0.0,2016,3.20,4.70,6.20,140.000000,267.000000,108.0,26.0
107,D.Cain,00-0034406,9.0,20.0,124.0,1.743712,0.0,0.283733,11.350000,2.444444,...,-1,-1.0,-1,-1.00,-1.00,-1.00,227.000000,0.000000,0.0,0.0
245,J.Holton,00-0032935,3.0,15.0,17.0,1.407284,0.0,0.336783,25.400000,2.000000,...,OAK,1.0,2016,5.70,6.70,7.70,381.000000,45.000000,413.0,35.0
431,R.Switzer,00-0033562,8.0,11.0,27.0,1.552952,0.0,0.369142,1.545455,2.125000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,17.000000,148.000000,33.0,0.0
462,T.Edmunds,00-0033838,5.0,6.0,41.0,0.848934,0.0,0.232555,0.500000,7.600000,...,-1,-1.0,-1,-1.00,-1.00,-1.00,3.000000,3.000000,0.0,0.0
